In [1]:
%load_ext autoreload
%autoreload 2

# Kvaser

> Kvaser class

In [2]:
#| default_exp dataflow.kvaser

In [3]:
#| export
import json
from threading import Event, current_thread
from typing import Optional, Tuple, cast
from dataclasses import dataclass
import pandas as pd

In [4]:
#| export
from data_io_nbdev.dataflow.consumer import Consumer  # type: ignore
from data_io_nbdev.dataflow.pipeline.queue import Pipeline  # type: ignore
from data_io_nbdev.dataflow.pipeline.deque import PipelineDQ  # type: ignore
from data_io_nbdev.dataflow.producer import Producer  # type: ignore
from data_io_nbdev.dataflow.vehicle_interface import VehicleInterface  # type: ignore

In [5]:
from data_io_nbdev.conn.tbox import TBoxCanException, kvaser_send_float_array
from data_io_nbdev.conn.udp import udp_context
from data_io_nbdev.data.core import MotionPower, RawType, KvaserType
from data_io_nbdev.config.messengers import CANMessenger, can_servers_by_name
from data_io_nbdev.config.vehicles import TruckInField

In [6]:
#| export
@dataclass
class Kvaser(VehicleInterface):
    """
    Kvaser is local vehicle interface with Producer(get vehicle status) and Consumer(flasher)
    
    Attributes:
        
        truck: TruckInField
            truck object
        can_server: CANMessenger
            can server object
    """
    truck: TruckInField
    can_server: CANMessenger = can_servers_by_name["can_udp_svc"]

    def __post_init__(self):
        super().__post_init__()
        self.logger.info("Kvaser initialized")

    def __str__(self):
        return "kvaser"

    def flash_vehicle(self, 
                    torque_table: pd.DataFrame  # the torque table to be flashed
                    ) -> None:
        """flash the torque table to the vehicle via kvaser"""
        
        try:
            kvaser_send_float_array(torque_table, sw_diff=True)
        except TBoxCanException as exc:
            raise exc
        except Exception as exc:
            raise exc

        self.logger.info(
            f"{{'header': 'Done with flashing table'}}",
            extra=self.dict_logger,
        )

    def init_internal_pipelines(
        self,
    ) -> Tuple[
        PipelineDQ[RawType], Pipeline[str]
    ]:  # Tuple[PipelineDQ[dict[str,Union[str,list[str]]]], Pipeline[str]]
        """initialize the internal pipelines for kvaser"""
        raw_pipeline = PipelineDQ[RawType](maxlen=1)
        hmi_pipeline = Pipeline[str](maxsize=1)
        return raw_pipeline, hmi_pipeline

    def produce(
        self,
        raw_pipeline: PipelineDQ[RawType],  # PipelineDQ[dict[str, str]],
        hmi_pipeline: Optional[Pipeline[str]] = None,  # HMI pipeline
        exit_event: Optional[Event] = None,  # input event exit
    ):
        """produce data from kvaser and put into the pipeline"""
        thread = current_thread()
        thread.name = "kvaser_capture"
        logger_kvaser_get = self.logger.getChild("kvaser_capture")
        logger_kvaser_get.propagate = True
        logger_kvaser_get.info(
            f"{{'header': 'kvaser capture thread start!'}}",
            extra=self.dict_logger,
        )

        with udp_context(self.can_server.host, self.can_server.port) as s:
            # self.logger.info('Data received!!!', extra=self.dict_logger)
            while not exit_event.is_set():
                try:
                    can_data, addr = s.recvfrom(2048)
                    pop_data = json.loads(can_data)
                except TypeError:
                    raise TypeError("udp sending wrong data type!")
                except Exception as exc:
                    logger_kvaser_get.error(
                        f"{{'header': 'udp reception error', " 
                        f"'exception': '{exc}'}}"
                    )
                    raise exc
                for key, value in pop_data.items():
                    if key == "status":  # state machine chores
                        assert (
                            type(value) is str
                        ), "udp sending wrong data type of status!"
                        hmi_pipeline.put_data(value)
                    elif key == "data":
                        assert (
                            type(value) is dict
                        ), "udp sending wrong data type of data!"
                        raw_pipeline.put_data(value)
                    else:
                        logger_kvaser_get.warning(
                            f"{{'header': 'udp sending message with key: {key}; value: {value}'}}"
                        )

                        break
                if (
                    key == "status" and value == "exit"
                ):  # exit thread and program, if earlier than the exit event
                    break

            # exit the thread
            logger_kvaser_get.info(
                f"{{'header': 'kvaser_capture dies!!!'}}", extra=self.dict_logger
            )

    def filter(
        self,
        in_pipeline: PipelineDQ[RawType],  # input PipelineDQ[dict[str, str]],
        out_pipeline: Pipeline[pd.DataFrame],  # output Pipeline[pd.DataFrame],
        start_event: Optional[Event],  # input event start
        stop_event: Optional[Event],  # input event stop
        interrupt_event: Optional[Event],  # input event interrupt
        flash_event: Optional[
            Event
        ],  # input & output event, maybe unnecessary for kvaser
        exit_event: Optional[Event],  # input event exit
    ) -> None:
        """filter data from kvaser input pipeline and put into the output pipeline"""
        thread = current_thread()
        thread.name = "kvaser_filter"
        logger_kvaser_out = self.logger.getChild("data_transform")
        logger_kvaser_out.propagate = True
        motion_power_t: list[MotionPower] = []
        logger_kvaser_out.info("{{'header': 'kvaser data transform thread start'}}", extra=self.dict_logger)

        while not exit_event.is_set():
            #  always get data from the pipeline if available, forwarding outward depends on the HMI status
            try:
                data: KvaserType = cast(
                    KvaserType, in_pipeline.get_data()
                )  # non-blocking, get the most recent data, cast is to sooth mypy

            except IndexError:
                continue  # empty deque

            # logger_kvaser_out.info("{{'header': 'kvaser get data'}}", extra=self.dict_logger)

            if start_event.is_set():  # starts episode
                try:
                    timestep = pd.Timestamp.now(tz=self.truck.site.tz)
                    velocity = float(data["velocity"])
                    pedal = float(data["pedal"])
                    brake = float(data["brake_pressure"])
                    current = float(data["A"])
                    voltage = float(data["V"])

                    motion_power = MotionPower(
                        timestep,
                        velocity,
                        pedal,
                        brake,
                        current,
                        voltage,
                    )
                    # 3 +2 : im 5

                    motion_power_t.append(
                        motion_power
                    )  # obs_reward [timestep, speed, pedal, brake, current, voltage]

                    if len(motion_power_t) == self.truck.observation_length:
                        df_motion_power = pd.DataFrame(
                            motion_power_t,
                            columns=[
                                "timestep",
                                "velocity",
                                "thrust",
                                "brake",
                                "current",
                                "voltage",
                            ],
                        )
                        # df_motion_power.set_index('timestamp', inplace=True)
                        df_motion_power.columns.name = "qtuple"

                        out_pipeline.put_data(df_motion_power)
                        logger_kvaser_out.info(
                            f"{{'header': 'convert one dataframe and wait.'}}",
                            extra=self.dict_logger,
                        )
                        flash_event.wait()  # wait for cruncher to consume and flashing to finish
                        flash_event.clear()  # clear the flash event here as the first waiter
                        logger_kvaser_out.info(
                            f"{{'header': 'wake up after flashing'}}",
                            extra=self.dict_logger,
                        )
                        # for kvaser the parameter is configured for waiting,
                        # maybe the event is not necessary, for cloud interface the flash event is necessary
                        observe_queue_size = out_pipeline.qsize()
                        if observe_queue_size != 0:
                            raise ValueError(
                                f"observe pipeline queue size: {observe_queue_size}, "
                                f"must be zero, if cruncher has consumed"
                            )
                        motion_power_t = []
                except Exception as exc:
                    logger_kvaser_out.info(
                        f"{{'header': 'kvaser get signal error',"
                        f"'exception': '{exc}'}}",
                        # f"Valid episode, Reset data capturing to stop after 3 seconds!",
                        extra=self.dict_logger,
                    )
                    break
            elif interrupt_event.is_set() or stop_event.is_set():  # interrupt episode
                motion_power_t = []  # clean up list of motion power states

        # exit the thread
        logger_kvaser_out.info(
            f"{{'header': 'data_transform dies!!!'}}", extra=self.dict_logger
        )

In [7]:
#| hide
from nbdev.showdoc import *

In [8]:
show_doc(Kvaser.flash_vehicle)
show_doc(Kvaser.init_internal_pipelines)
show_doc(Kvaser.produce)
show_doc(Kvaser.filter)

---

### Kvaser.filter

>      Kvaser.filter (in_pipeline:data_io_nbdev.dataflow.pipeline.deque.Pipeline
>                     DQ[typing.Union[dict[str,str],dict[str,dict[str,list[typin
>                     g.Union[str,list[list[str]]]]]]]], out_pipeline:data_io_nb
>                     dev.dataflow.pipeline.queue.Pipeline[pandas.core.frame.Dat
>                     aFrame], start_event:Optional[threading.Event],
>                     stop_event:Optional[threading.Event],
>                     interrupt_event:Optional[threading.Event],
>                     flash_event:Optional[threading.Event],
>                     exit_event:Optional[threading.Event])

filter data from kvaser input pipeline and put into the output pipeline

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| in_pipeline | PipelineDQ | input PipelineDQ[dict[str, str]], |
| out_pipeline | Pipeline | output Pipeline[pd.DataFrame], |
| start_event | Optional | input event start |
| stop_event | Optional | input event stop |
| interrupt_event | Optional | input event interrupt |
| flash_event | Optional |  |
| exit_event | Optional | input event exit |
| **Returns** | **None** |  |

In [9]:
#| hide
import nbdev; nbdev.nbdev_export()